In [2]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx

In [3]:
node = pd.read_csv('../../data/shop/daejeon_node.csv', encoding='utf-8')
edge = pd.read_csv('../../data/shop/daejeon_edge.csv', encoding='utf-8')

## 데이터 가중치 부여(연결중심성)

In [4]:
G = nx.Graph()


for idx,row in node.iterrows():

    G.add_node(row['node_name'],Label=row['node_name'],latitude=row['lat'], longitude=row['lon'])


# 1. 노드와 연결된 노드의 숫자
# 2. 중심성이 높은 노드와 연결된 갯수로 가중치를 넣는다
for idx,row in edge.iterrows():
    # 연결이 1개당 가중치 1씩 증가하는 형태
    G.add_edge(row['start'],row['end'],weight = 1)

## 데이터 가중치 부여(위세중심성)

In [5]:
avg = []

for idx, row in node.iterrows():
    # row['node_name']을 통해서 row['node_name']이 가진 연결점을 호출 
    link = edge[edge['start'] == row['node_name']]['end']
    sum = 0
    
    # 연결된 노드들의 가중치를 평균내서 위세중심성을 추측
    for i in link:
        sum += G.degree[i]
    a = sum/(len(link)+1)
    avg.append(sum)

In [6]:
node['w'] = (avg-np.min(avg))/(np.max(avg)-np.min(avg))

## 데이터 minmax정규화

In [7]:
# G.degree 에 대한 max,min을 추출해야함 min은 0으로 확정
# max를 구해야함
a = 0
for idx, row in node.iterrows():
    
    if a < G.degree[row['node_name']]:
        a = G.degree[row['node_name']]

In [ ]:
# max degree = 136

node[node['w'] == node['w'].max()]

In [16]:
a

14

In [9]:
G.degree
std_point = tuple(node.head(1)[['lat','lon']].iloc[0])

## 시각화(연결중심성)

In [17]:
map_osm = folium.Map(location=std_point, zoom_start=10) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

for idx, row in node.iterrows():
    
    w = G.degree[row['node_name']]/136
    
    location = (row['lat'], row['lon']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=G.degree[row['node_name']]/14*1000+1,# 지름이 degree에 비례하도록 설정
        color='red',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        popup=str(row['node_name'])
    ).add_to(map_osm)

map_osm

In [18]:
map_osm = folium.Map(location=std_point, zoom_start=10) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

for idx, row in node.iterrows():
    location = (row['lat'], row['lon']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=row['w']*1000+1,# 지름이 degree에 비례하도록 설정
        color='red',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True, 
        popup=str(row['node_name'])
    ).add_to(map_osm)
map_osm

In [19]:
# 가장 연결중심성이 높은 위치의 ID(위경도)를 추출
l = list(G.degree)
d = pd.DataFrame()

for i in G.degree:
    d.loc[i[0],'count'] = i[1]

d1 = d.sort_values(by=['count'])
i_l = list(d1.tail(10).index)
i_l

['36.36267470000001,127.35142590000001',
 '36.3484097,127.38984599999999',
 '36.398962100000006,127.40114380000001',
 '36.3994427,127.40110940000001',
 '36.3993934,127.4019221',
 '36.33558970000001,127.4463511',
 '36.3387756,127.4492683',
 '36.327116700000005,127.42152890000001',
 '36.332326,127.434211',
 '36.3504119,127.38454750000001']

In [20]:
# 위세중심성의 가중치 높은 10개를 추출
n_l = node.sort_values(by=['w']).tail(10).index

i_l1 =  list(node.iloc[n_l,:]['node_name'])

print(i_l1)

['36.3484097,127.38984599999999', '36.398962100000006,127.40114380000001', '36.3994427,127.40110940000001', '36.3993934,127.4019221', '36.355593,127.33472250000001', '36.362216,127.35613290000002', '36.332326,127.434211', '36.3609648,127.3762836', '36.36267470000001,127.35142590000001', '36.3504119,127.38454750000001']


In [21]:
for i in range(9,-1,-1):
    for j in range(0,10):
        if i_l[i] == i_l1[j]:
            print(i_l[i], i, j)

      
      

36.3504119,127.38454750000001 9 9
36.332326,127.434211 8 6
36.3993934,127.4019221 4 3
36.3994427,127.40110940000001 3 2
36.398962100000006,127.40114380000001 2 1
36.3484097,127.38984599999999 1 0
36.36267470000001,127.35142590000001 0 8


In [22]:
for i in range(0,5):
    print(i_l[i], i)
    print(i_l1[i], i)
    

36.36267470000001,127.35142590000001 0
36.3484097,127.38984599999999 0
36.3484097,127.38984599999999 1
36.398962100000006,127.40114380000001 1
36.398962100000006,127.40114380000001 2
36.3994427,127.40110940000001 2
36.3994427,127.40110940000001 3
36.3993934,127.4019221 3
36.3993934,127.4019221 4
36.355593,127.33472250000001 4


In [23]:
df = pd.read_csv('../../data/shop/PatientRoute.csv',encoding = 'utf-8')    

In [24]:
for i in range(0,10):
    lat = float(i_l1[i].split(',')[0])
    lon = float(i_l1[i].split(',')[1])
    print(df[(df['latitude'] == lat)&(df['longitude'] == lon)]['city'].iloc[0])
                

Seo-gu
Yuseong-gu
Yuseong-gu
Yuseong-gu
Yuseong-gu
Yuseong-gu
Dong-gu
Seo-gu
Yuseong-gu
Seo-gu


kw = {'opacity': 0.5, 'weight': 1}
for ix, row in edge.iterrows():
    start = tuple(node[node['node_name']==row['start']][['lat','lon']].iloc[0])
    end = tuple(node[node['node_name']==row['end']][['lat','lon']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        width = 0.01,
     
        **kw,
    ).add_to(map_osm)

map_osm

# 심심해서 구별로 종합

In [34]:
df = pd.read_csv('../../data/shop/PatientRoute.csv', encoding = 'utf-8')

In [38]:
df_s = df[df['province'] == 'Daejeon']
gu_list = df_s['city'].unique()

In [ ]:
kgu_list = ['중구','성북구','성동구','동대문구','중랑구','강남구','종로구','송파구'
           ,'서대문구','마포구','은평구','서초구','광진구','구로구','도봉구','동작구',
           '금천구','영등포구','강동구','노원구','양천구','용산구','강서구','관악구','경북구']
df_s['구군'] = ''
df_s['node_name'] = ''

In [39]:
kgu_list = ['서구','중구','동구','유성구','대덕구']
df_s['구군'] = ''
df_s['node_name'] = ''

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
for i, j in zip(gu_list, kgu_list):
    for z in df_s[df_s['city'] == i].index:

        df_s.loc[z,'구군'] = j
        a = df_s.loc[z,'latitude'].astype(str)
        b = df_s.loc[z,'longitude'].astype(str)
        c = a+','+b
        df_s.loc[z,'node_name'] = c

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [41]:
node['구군'] = ''

for i in node.index:
    a = node.loc[i,'node_name']
    b = df_s[df_s['node_name'] == a]['구군'].values[0]
    node.loc[i,'구군'] = b
    

In [42]:
node

,id,node_name,lat,lon,w,구군
0,0,"36.358111799999996,127.3354294",36.358112,127.335429,0.000000,서구
1,1,"36.3504119,127.38454750000001",36.350412,127.384547,1.000000,중구
2,2,"36.3612482,127.3848091",36.361248,127.384809,0.000000,중구
3,3,"36.31695,127.41572079999999",36.316950,127.415721,0.000000,동구
4,4,"36.355593,127.33472250000001",36.355593,127.334722,0.124224,서구
5,5,"36.362216,127.35613290000002",36.362216,127.356133,0.124224,서구
6,6,"36.3253401,127.42447790000001",36.325340,127.424478,0.000000,동구
7,7,"36.35165670000001,127.4374114",36.351657,127.437411,0.086957,유성구
8,8,"36.332326,127.434211",36.332326,127.434211,0.273292,유성구
9,9,"36.298141799999996,127.3372517",36.298142,127.337252,0.000000,중구


In [43]:
node['w1'] = avg
node_name = list(node['node_name'].unique())
weight = []
for i in node_name:
    a = G.degree(i)
    weight.append(a) 
node['w'] = weight    

In [44]:
gu_mean = node.groupby('구군').mean()
gu_mean = gu_mean.reset_index()

In [45]:
a = list(gu_mean['w'])
gu_mean['w'] = (a-np.min(a))/(np.max(a)-np.min(a))
a = list(gu_mean['w1'])
gu_mean['w1'] = (a-np.min(a))/(np.max(a)-np.min(a))
gu_mean

,구군,id,lat,lon,w,w1
0,동구,13.000000,36.324261,127.422380,0.000000,0.000000
1,서구,10.333333,36.380945,127.366314,0.661376,0.319088
2,유성구,14.400000,36.338575,127.444015,1.000000,0.335043
3,중구,7.200000,36.343835,127.374548,0.809524,1.000000


In [50]:
std_point = tuple(node.head(1)[['lat','lon']].iloc[0])
map_osm = folium.Map(location=std_point, zoom_start=11) 
for idx, row in gu_mean.iterrows():
        
    location = (row['lat'], row['lon']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=row['w']*2000+1,# 지름이 degree에 비례하도록 설정
        color='red',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        popup=str(row['구군'])
    ).add_to(map_osm)
map_osm.save('../../data/shop/서울연결.html')

In [51]:
std_point = tuple(node.head(1)[['lat','lon']].iloc[0])
map_osm = folium.Map(location=std_point, zoom_start=11) 
for idx, row in gu_mean.iterrows():
        
    location = (row['lat'], row['lon']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=row['w1']*2000+1,# 지름이 degree에 비례하도록 설정
        color='red',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        popup=str(row['구군'])
    ).add_to(map_osm)
map_osm.save('../../data/shop/서울위세.html')

In [49]:
# 평균으로 만들 시 가중치의 평균이 계산에 맞는가?
gu_mean.to_csv('../../data/shop/서울구별가중치평균.csv',index=False, sep=',', encoding = 'utf-8')
gu_mean.to_csv('../../data/shop/서울구별가중치평균_R.csv',index=False, sep=',', encoding = 'ANSI')

## 정리본인듯


In [ ]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx

node = pd.read_csv('../../data/shop/deagu_node.csv', encoding='utf-8')
edge = pd.read_csv('../../data/shop/deagu_edge.csv', encoding='utf-8')

G = nx.Graph()


for idx,row in node.iterrows():
    
    G.add_node(row['node_name'],Label=row['node_name'],latitude=row['lat'], longitude=row['lon'])

# 예제에서는 거리에 비례해서 가중치를 넣음
# 1. 노드와 연결된 노드의 숫자
# 2. 중심성이 높은 노드와 연결된 갯수로 가중치를 넣는다
for idx,row in edge.iterrows():
    # 연결이 1개당 가중치 1씩 증가하는 형태
    G.add_edge(row['start'],row['end'],weight = 1)

avg = []
# 예제에서는 거리에 비례해서 가중치를 넣음
# 1. 노드와 연결된 노드의 숫자
# 2. 중심성이 높은 노드와 연결된 갯수로 가중치를 넣는다
for idx, row in node.iterrows():
    # row['node_name']을 통해서 row['node_name']이 가진 연결점을 호출 
    link = edge[edge['start'] == row['node_name']]['end']
    sum = 0
    for i in link:
        sum += G.degree[i]
    a = sum/(len(link)+1)
    avg.append(sum)

node['w'] = (avg-np.min(avg))/(np.max(avg)-np.min(avg))

a = 0
for idx, row in node.iterrows():
    
    if a < G.degree[row['node_name']]:
        a = G.degree[row['node_name']]

G.degree
std_point = tuple(node.head(1)[['lat','lon']].iloc[0])

In [28]:
map_osm = folium.Map(location=std_point, zoom_start=10) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

for idx, row in node.iterrows():
    
    w = G.degree[row['node_name']]/a
    
    location = (row['lat'], row['lon']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=G.degree[row['node_name']]/a*1000+1,# 지름이 degree에 비례하도록 설정
        color='red',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        #popup=str(row['node_name'])
    ).add_to(map_osm)
    # folium.Marker(location, popup=row['NODE_NAME']).add_to(map_osm)
map_osm

In [1]:
map_osm = folium.Map(location=std_point, zoom_start=10) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

for idx, row in node.iterrows():
    location = (row['lat'], row['lon']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=row['w']*100+1,# 지름이 degree에 비례하도록 설정
        color='red',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        popup=str(row['node_name'])
    ).add_to(map_osm)
    
map_osm

NameError: name 'folium' is not defined

In [ ]:
전체
 
구 한개만 - 각각 볼때 
x1 코로나자수
x2 경제성장률 
y 데이터 
구별경제지표
상점에 영향을 주는 지표
1. 유동인구수
2. 교통량 - 버스정류장수, 지하철수, 버스노선의 수
3. 유흥업소 수
4. 매출x, 세금x, 동종업종수, 일반음식점수
5. 
6. 
